In [42]:
# [1] 코스피 지수 구성종목 분석 페이지를 시가총액 순으로 긁어오자.

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

def create_analysis_link(link):
    new = 'https://finance.naver.com' + link
    return 'https://finance.naver.com' + link[:6] + 'coinfo' + link[10:]


links = [] # 최종적인 링크들을 담을 테이블

# 시가총액 1~200위 페이지
for page in range(1,5):
    html = urlopen('https://finance.naver.com/sise/sise_market_sum.naver?&page='+str(page))
    bs = BeautifulSoup(html, 'html.parser')
    rankings = bs.findAll('a',{'href' : re.compile('^(/item/main.)')})

    # 각 종목별 페이지로 연결
    for ranking in rankings:
            if 'href' in ranking.attrs:
                midLink = str(create_analysis_link(ranking.attrs['href']))
                
                # 종목별 페이지 속의 '재무분석' 페이지로 연결
                html = urlopen("{}".format(midLink))
                bs = BeautifulSoup(html,'html.parser')
                link = bs.find('div', {'class' : 'section inner_sub'}).find('iframe')
                
                if 'src' in link.attrs:
                    financialLink = link.attrs['src']
                    links.append(financialLink)
                    
print(links[140])

https://navercomp.wisereport.co.kr/v2/ETF/index.aspx?cmp_cd=133690


In [127]:
# [2] 어떤 자료들을 긁어올 것인지 '크롤링 모델'을 설정하자.

def unit_conversion(num):
    return round(float(num.replace(",","")))*10**8

class Content:
    
# bs_arr = [title, [ EPS, BPS, PER, 업종PER, PBR, 현금배당수익률]]
# sel_arr = [market_cap, per_arr, current_asset, current_liability, cashable_asset]
    
    # 데이터 입력받기
    def __init__(self, bs_arr):
 
        # BeautifulSoup으로 긁어올 정보들 (추후에 정리)
        self.title = bs_arr[0]
        self.eps = bs_arr[1][0]
        self.bps = bs_arr[1][1]
        self.per = bs_arr[1][2]
        self.per_industry = bs_arr[1][3]
        self.pbr = bs_arr[1][4]
        self.dividend_rate = bs_arr[1][5]
    
    def input_sel(self, sel_arr):
    
        # Selenium으로 긁어올 정보들 (추후에 정리)
        self.market_cap = sel_arr[0]
        self.per_average = sel_arr[1]
        self.current_asset = sel_arr[2]
        self.current_liability = sel_arr[3]
        self.cashable_asset = sel_arr[4]
        
    # return
    def return_data(self):
        data = [self.title, self.market_cap, self.per, self.per_average, self.per_industry, self.dividend_rate,
        self.current_asset, self.current_liability, self.cashable_asset]
        return data
                    

In [134]:
# [3] BeautifulSoup으로 긁어오기.

def make_name(arr):
    return arr[6][12:] + ' ' + 'ETF'

def extract_name(name):
    i = -1
    for i in range(1, len(name)):
        if name[-i] == '(':
            name = name[-i+1:-1]
            break
    return name

# writer.writerow(('종목 이름', 'EPS', 'BPS', 'PER', '업종PER', 'PBR', '현금배당수익률'))
# rank, title, basic data(리스트)

finance_data = [0] * 201

i=1
for one in links[:10]:
    html = urlopen("{}".format(one))
    bs = BeautifulSoup(html, 'html.parser')
    try:
        # title 가져오기
        title = bs.find('title').get_text()
        title = extract_name(title) # 형식에 맞게 수정
            
        # 재무정보 가져오기
        table = bs.findAll('table',{'class' : 'cmp-table'})[0]
        rows = table.findAll('tr')[2].findAll('dt')

        data = []
        for row in rows:
                
        # 필요없는 정보면 continue
            if row.get_text() == '12월 결산': 
                continue
                    
            temp = row.get_text().split()
            if len(temp) == 1:
                temp.append('N/A')
            data.append(temp[1])
                
    # ETF일 때.        
    except IndexError as e:
        bs = bs.findAll('script', {'type':'text/javascript'})[2]
        arr = list(map(str,str(bs).split()))
            
        title = 'ETF'
        data = ['N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
        
    finally:
        bs_arr = [title, data]
        content = Content(bs_arr)
        finance_data[i] = content
        i += 1

In [135]:
# [4] Selenium으로 긁어오기.

from selenium import webdriver
from selenium.webdriver.common.by import By

import time

# headless 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')

i=1
for one in links[:10]:
    try:
        # 브라우저 생성
        browser = webdriver.Chrome(chrome_options=options)
        browser.get('{}'.format(one))

        # 시가총액 구하기
        elementXpath = '//*[@summary="기업의 기본적인 시세정보(주가/전일대비/수익률,52주최고/최저,액면가,거래량/거래대금,시가총액,유동주식비율,외국인지분율,52주베타,수익률(1M/3M/6M/1Y))를 제공합니다."]/tbody/tr[5]/td'
        market_cap = browser.find_element(by=By.XPATH, value = elementXpath).text

        # per 평균 구하기
        browser.find_element(by=By.XPATH, value = '//*[@title="연간"]').click()

        elementXpath = '//table[@summary="주요재무정보를 제공합니다."][2]/tbody/tr[27]'
        element = browser.find_element(by=By.XPATH, value = elementXpath)
        per_table = [float(x) for x in element.text.split()[1:6] if x != 'N/A']
        per_arr = round(sum(per_table)/len(per_table), 2)

        # 유동자산/유동부채 구하기
        browser.find_element_by_link_text("재무분석").click()
        browser.find_element_by_link_text("재무상태표").click()

        # Loading
        time.sleep(1)

        # 참고로 Xpath는 시작점이 0이 아니라 1이다.
        try:
            elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[2]/td[6]'
            current_asset = browser.find_element(by=By.XPATH, value = elementXpath).text
        except:
            current_asset = 'N/A'

        try:
            elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[128]/td[6]'
            current_liability = browser.find_element(by=By.XPATH, value = elementXpath).text
        except:
            current_liability = 'N/A'

        # 현금 및 현금성 자산 구하기
        try:
            elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[47]/td[6]'
            cashable_asset = browser.find_element(by=By.XPATH, value = elementXpath).text
        except:
            cashable_asset = 'N/A'

        # 객체화
        sel_arr = [market_cap, per_arr, current_asset, current_liability, cashable_asset]
        
    # ETF일 때. 
    except: 
        sel_arr = ['N/A', 'N/A', 'N/A', 'N/A', 'N/A']

    finally:
        finance_data[i].input_sel(sel_arr)
        finance_data[i] = finance_data[i].return_data()
        browser.quit()
        i+=1

        
"""
#. return_data()의 성질
[self.title, self.market_cap, self.per, self.per_average, self.per_industry, self.dividend_rate,
        self.current_asset, self.current_liability, self.cashable_asset]
"""
for data in finance_data[1:11]:
    print(data)


C:\Users\dck\AppData\Local\Temp/ipykernel_18408/3285596659.py:16: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=options)
C:\Users\dck\AppData\Local\Temp/ipykernel_18408/3285596659.py:32: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  browser.find_element_by_link_text("재무분석").click()
C:\Users\dck\AppData\Local\Temp/ipykernel_18408/3285596659.py:33: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  browser.find_element_by_link_text("재무상태표").click()


['삼성전자', '4,125,120억원', '11.96', 13.62, '10.64', '2.09%', '2,181,631.9', '881,171.3', '390,314.2']
['LG에너지솔루션', '1,001,520억원', '108.01', 89.9, '77.54', 'N/A', 'N/A', 'N/A', 'N/A']
['SK하이닉스', '844,483억원', '8.79', 14.06, '10.64', '1.33%', '268,704.5', '147,693.6', '50,579.8']
['NAVER', '550,385억원', '3.34', 34.51, '5.94', '0.15%', '55,278.8', '39,233.2', '27,814.0']
['삼성바이오로직스', '542,553억원', '139.51', 267.52, '83.29', 'N/A', 'N/A', 'N/A', 'N/A']
['삼성전자', '4,125,120억원', '11.96', 13.62, '10.64', '2.09%', '2,181,631.9', '881,171.3', '390,314.2']
['카카오', '472,890억원', '33.84', 127.17, '5.94', '0.05%', '81,021.9', '52,461.2', '52,315.0']
['삼성SDI', '403,648억원', '35.32', 41.36, '77.54', '0.17%', '74,449.1', '64,612.9', '23,256.9']
['현대차', '380,329억원', '9.97', 18.68, '7.31', '2.81%', '533,127.6', '642,367.9', '127,955.5']
['LG화학', '369,198억원', '11.16', 50.6, '8.63', '2.29%', '204,138.2', '150,621.0', '37,608.3']


In [142]:
import csv

csvFile = open('테스트파일.csv', 'wt+', newline = '')
writer = csv.writer(csvFile)

writer.writerow(('기업 이름', '시가총액(억)', 'PER', '5년 평균 PER', '업종 평균 PER',
                '현금배당수익률','유동자산(억)','유동부채(억)','현금 및 현금성 자산(억)'))

try:
    for one in finance_data[1:11]:
        writer.writerow(one)
        
finally:
    csvFile.close()

In [ ]:
"""
매우 뿌듯하다. 
수 많은 에러와 역경을 뚫고 원하던 결과물을 냈다...
물론 부족한 점이 너무 많다.

(1) 클래스가 너무 잡다해서 다른 사람이 알아볼 수가 없다.
(2) 클래스 뿐 아니라 변수명이나 구조도 너무 복잡해서 다른 사람이 읽을 수가 없다. 주석도 필요하다.
(3) 셀레니움 크롤링은 너무 오래걸린다.
단순히 셀레니움만 쓸 것이 아니라, 셀레니움으로 띄우고 뷰티풀솦으로 크롤링할 방법을 찾아야 한다. 
몇 배는 빨라질 것이다. 

하지만 문제는 '재무상태표'를 클릭한 상태로 html 파일을 뷰티풀 솦으로 보내도 이상하게 
해당 html은 '재무상태표'가 아니라 '포괄손익계산서'가 담긴 html을 띄운다.. 어찌된 일일까?
좀 뒤적여본 결과, 아마도 자바스크립트 상의 내용인 것 같다. 

대충 보더라도 해결해야할 부분이 3가지이다.
우선 오늘밤에 한번 200위까지 크롤링을 돌려보고 다시 한번 리뷰를 해보도록 하겠다.
"""
